In [ ]:
import requests

url = 'https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVINValuesBatch/';

vinnumber = 'NM0LS7CN2CT110748'
post_fields = {'format': 'json', 'data':vinnumber};

response = requests.post(url, data=post_fields);

print(response.text);	

In [25]:
import pandas as pd
import requests

# Step 1: Read from CSV
csv_file = 'E:\\vin_data.csv' 
vin_data = pd.read_csv(csv_file)
vin_data.head(3)

,sf_code,vin
0,a0A5d00000xi1COEAY,NM0LS7CN2CT110748
1,a0A5d00000xiG74EAE,1GCEC14X75Z255824
2,a0A5d00000xiGO7EAM,1GBGK24R1VZ212782


In [26]:
def get_vin_info(vin):
    url = 'https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVINValuesBatch/'
    post_fields = {'format': 'json', 'data': vin}
    
    try:
        response = requests.post(url, data=post_fields)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
        vin_info = response.json()

        # Extract information from JSON response and save it in a dictionary
        if 'Results' in vin_info:
            result = vin_info['Results'][0]
            info_dict = {
                'VIN': result.get('VIN', ''),
                'Make': result.get('Make', ''),
                'Model': result.get('Model', ''),
                'Year': result.get('ModelYear', ''),
                'BodyClass': result.get('BodyClass', ''),
                'VehicleType': result.get('VehicleType', ''),
                'GVWR': result.get('GVWR', '')
            }

            return info_dict

    except requests.exceptions.RequestException as req_ex:
        print(f"Error processing VIN {vin}: {req_ex}")
        return None

In [27]:
result_list = []

for index, row in vin_data.iterrows():
    vin = row['vin']
    policy_number = row['sf_code']
    
    vin_info = get_vin_info(vin)
    
    if vin_info:
        result_list.append(vin_info)

#result_list

In [59]:
# Create a DataFrame from the result list
result_df = pd.DataFrame(result_list)
final_result = result_df.query('Make != "" and Model != "" ')
final_result.head(2)

,VIN,Make,Model,Year,BodyClass,VehicleType,GVWR
0,NM0LS7CN2CT110748,FORD,Transit Connect,2012,Van,TRUCK,"Class 1D: 5,001 - 6,000 lb (2,268 - 2,722 kg)"
1,1GCEC14X75Z255824,CHEVROLET,Silverado,2005,Pickup,TRUCK,"Class 2E: 6,001 - 7,000 lb (2,722 - 3,175 kg)"


In [54]:
# Group by 'Make' and get counts
make_counts = result_df.groupby('Make').size().reset_index(name='Make_Count')

# Group by 'Model' and get counts
model_counts = result_df.groupby('Model').size().reset_index(name='Model_Count')

# Group by 'BodyClass' and get counts
body_class_counts = result_df.groupby('BodyClass').size().reset_index(name='BodyClass_Count')

# Group by 'VehicleType'
vehicle_type_counts = result_df.groupby('VehicleType').size().reset_index(name='vehicle_type_count')

# Group by 'VehicleType'
GVWR_counts = result_df.groupby('GVWR').size().reset_index(name='GVWR_count')

In [57]:
sorted_bodyclass = body_class_counts.sort_values(by='BodyClass_Count', ascending=False)
GVWR_counts.sort_values(by='GVWR_count', ascending=False)

,GVWR,GVWR_count
6,"Class 2E: 6,001 - 7,000 lb (2,722 - 3,175 kg)",63
4,"Class 1D: 5,001 - 6,000 lb (2,268 - 2,722 kg)",55
0,,51
8,"Class 2G: 8,001 - 9,000 lb (3,629 - 4,082 kg)",48
9,"Class 2H: 9,001 - 10,000 lb (4,082 - 4,536 kg)",47
10,"Class 3: 10,001 - 14,000 lb (4,536 - 6,350 kg)",30
7,"Class 2F: 7,001 - 8,000 lb (3,175 - 3,629 kg)",23
11,"Class 4: 14,001 - 16,000 lb (6,350 - 7,258 kg)",22
3,"Class 1C: 4,001 - 5,000 lb (1,814 - 2,268 kg)",21
2,"Class 1B: 3,001 - 4,000 lb (1,360 - 1,814 kg)",3


In [64]:
final_result.dtypes

VIN            object
Make           object
Model          object
Year           object
BodyClass      object
VehicleType    object
GVWR           object
dtype: object

In [68]:
final_result['Year'] = final_result['Year'].astype(int)

C:\Users\marcorob\AppData\Local\Temp\ipykernel_27196\592757170.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_result['Year'] = final_result['Year'].astype(int)


In [69]:
final_result.dtypes

VIN            object
Make           object
Model          object
Year            int32
BodyClass      object
VehicleType    object
GVWR           object
dtype: object

In [72]:
final_result.to_excel('E:\\transformed_data.xlsx', index=False)